# Downloading VIIRS NDVI

VNP13A1 https://developers.google.com/earth-engine/datasets/catalog/NOAA_VIIRS_001_VNP13A1#description

## Load packages

In [ ]:
import os
import ee
import eemont
import geemap
import wxee
import math
import warnings
import xarray as xr
import pandas as pd
import numpy as np
from odc.geo.xr import assign_crs

## Authenticate GEE    

In [ ]:
# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize()

## Analysis parameters


In [ ]:
product = 'NOAA/VIIRS/001/VNP13A1' 
bands =  ['NDVI']
start_date = '1/1/2012'
end_date = '12/31/2022'
output_crs = "EPSG:3577"
resolution = 5000

region = ee.Geometry.Polygon([[
            [112,-43.0],
            [155,-43.0],
            [155,-9.0],
            [112,-9.0],
            [112,-43.0]]])


In [ ]:
# def getQABits(image, start, end, newName):
#     # Compute the bits we need to extract.
#     pattern = 0
#     for i in range(start, end, 1):
#        pattern += math.pow(2, i)

#     out = (image.select([0], [newName])
#                   .bitwiseAnd(pattern)
#                   .rightShift(start))
#     return(image.addBands(out))

# def qualityFlag(image):
#     qa = getQABits(image.select('QC'), 0,1,"good").eq(0)
#     return image.updateMask(qa.select('good'))

# def qualityFlagMOD11A1(image):
#   qa = image.select('QC')
#   bitMask2 = 1 << 2
#   bitMask3 = 1 << 3
#   mask = qa.bitwiseAnd(bitMask2).eq(0).And(qa.bitwiseAnd(bitMask3).eq(0))
#   return image.updateMask(mask)

In [ ]:
#use start and end dates to loop through months and load GEE data
start = pd.date_range(start=start_date, end=end_date, freq='MS') 
end = pd.date_range(start=start_date, end=end_date, freq='M')

i = 0
for s, e in zip(start,end):
    print(" {:03}/{:03}\r".format(i+1, len(start)), end="")

    ss = s+pd.Timedelta(14, 'd')

    s = s.strftime('%Y-%m-%d')
    e = e.strftime('%Y-%m-%d')

    ds = (ee.ImageCollection(product)
      .filterDate(s,e)
      .map(qualityFlag)
      .map(lambda image: image.clip(region))
      .select(bands)
      # .reduce(ee.Reducer.count())  
      # .reduce(ee.Reducer.stdDev())
      .median()
         )

    export = ee.batch.Export.image.toDrive(ds,
        description='VIIRS_NDVI'+'_'+ss.strftime('%Y-%m-%d'),
        folder='VIIRS_NDVI_VNP13A1_5km',
        region=region,
        scale=resolution,
        crs=output_crs,
        maxPixels=1e13,
        fileFormat='GeoTIFF',
        formatOptions={
            'cloudOptimized':True
              }
        )
    export.start()
    
    i+=1
    

In [ ]:
export.status()

In [ ]:
Map = geemap.Map()
visParams = {'min':280, 'max':320, 'palette': ['green', 'yellow', 'red']}

Map.addLayer(ds,visParams,'LST')
Map

## Stitch together geotiffs

In [ ]:
import os
import xarray as xr
import numpy as np
import pandas as pd
from odc.geo.xr import assign_crs

In [ ]:
# base='/g/data/os22/chad_tmp/climate-carbon-interactions/data/MODIS_LST/'
base='/g/data/os22/chad_tmp/climate-carbon-interactions/data/MODIS_NDVI_median/'
name= 'LST_median'
dates_start = '2000-03-01'
dates_end = '2000-12-31'


In [ ]:
files = [base+i for i in os.listdir(base) if i.endswith('.tif')]
files.sort()

time = (pd.date_range(dates_start, dates_end, freq='MS') + pd.DateOffset(days=14)).to_numpy()
time = list(time)

print(len(files), print(len(time)))

In [ ]:
i=0
arrs=[]
for f, t in zip(files, time):
    print(" {:02}/{:02}\r".format(i + 1, len(files)), end="")
    
    ds = xr.open_dataarray(f).squeeze().drop('band').astype('float32')
    ds = ds.expand_dims(time=[t])
    ds = ds.rename(name)
    del ds.attrs['long_name']
    del ds.attrs['AREA_OR_POINT']

    ds.attrs['nodata']=np.nan
    ds = ds.drop('spatial_ref')
    ds = assign_crs(ds, crs='epsg:3577')
    arrs.append(ds)

    i+=1 

In [ ]:
ds = xr.concat(arrs, dim='time').sortby('time')

In [ ]:
# ds.plot.imshow(col='time')

In [ ]:
gbox = xr.open_dataset('/g/data/os22/chad_tmp/climate-carbon-interactions/data/5km/WCF_5km_monthly_1982_2022.nc').odc.geobox

In [ ]:
ds = ds.odc.reproject(how=gbox, resampling='bilinear').compute()

In [ ]:
dss = assign_crs(xr.open_dataset('/g/data/os22/chad_tmp/climate-carbon-interactions/data/LST_harmonization/MODIS_LST_5km_monthly_2001_2022.nc'), crs='epsg:4326')

In [ ]:
out = xr.concat([ds.to_dataset(name='LST_median'), dss], dim='time').sortby('time')

In [ ]:
out.to_netcdf('/g/data/os22/chad_tmp/climate-carbon-interactions/data/LST_harmonization/MODIS_LST_5km_monthly_20003_202212.nc')